# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></div><div class="lev1 toc-item"><a href="#Define-loss-functions" data-toc-modified-id="Define-loss-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define loss functions</a></div><div class="lev1 toc-item"><a href="#Define-models" data-toc-modified-id="Define-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define models</a></div><div class="lev1 toc-item"><a href="#Training" data-toc-modified-id="Training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training</a></div><div class="lev1 toc-item"><a href="#Predictions" data-toc-modified-id="Predictions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Predictions</a></div>

Modified from https://github.com/petrosgk/Kaggle-Carvana-Image-Masking-Challenge

# Load libraries

In [1]:
import cv2
import numpy as np
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Define loss functions

In [2]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + (1 - dice_loss(y_true, y_pred))

# Define models

In [3]:
def unet_down_one_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [4]:
def unet_max_pool(inputs):
    x = MaxPooling2D((2, 2), strides=(2, 2))(inputs)
    return x

In [5]:
def unet_up_one_block(up_input, down_input, num_filters):
    x = UpSampling2D((2,2))(up_input)
    x = concatenate([down_input, x], axis=3)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [6]:
def get_unet(input_shape = (256, 256, 3),
             num_classes = 1,
             initial_filters = 32,
             central_filters = 1024):
    
    num_filters = initial_filters
    
    out_list    = [Input(shape=input_shape)]
    down_interim_list = []
    
    while num_filters <= central_filters/2:
        x = unet_down_one_block(out_list[-1], num_filters)
        down_interim_list.append(x)
        num_filters = num_filters * 2
        y = unet_max_pool(x)
        out_list.append(y)
    
    x = unet_down_one_block(out_list[-1], num_filters)
    out_list.append(x)
    num_filters = int(num_filters / 2)
    
    while num_filters >= initial_filters:
        x = unet_up_one_block(out_list[-1], down_interim_list.pop(), num_filters)
        out_list.append(x)
        num_filters = int(num_filters / 2)
    
    classify = Conv2D(num_classes, (1,1), activation = 'sigmoid')(out_list[-1])
    
    model = Model(inputs=out_list[0], outputs=classify)
    
    model.compile(optimizer=RMSprop(lr=0.0001),
                  loss=bce_dice_loss,
                  metrics=[dice_loss])
    
    return model

In [7]:
model = get_unet(input_shape=(512,512,3), initial_filters=16)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512, 512, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 512, 512, 16)  448         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 512, 512, 16)  64          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 512, 512, 16)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [8]:
from keras.utils import plot_model  
plot_model(model, to_file='model-512.png', show_shapes=True, show_layer_names=True)  
from IPython.display import FileLink
FileLink('model-512.png')

/home/ubuntu/extvol/kaggle-carvana-image-masking-challenge/model-512.png

# Training

In [8]:
input_size = 512
max_epochs = 50
orig_width = 1918
orig_height= 1280
threshold  = 0.5

train_batch_size = 14
val_batch_size   = 32

In [9]:
df_train = pd.read_csv('data/train_masks.csv')

In [10]:
df_train.head()

,img,rle_mask
0,00087a6bd4dc_01.jpg,879386 40 881253 141 883140 205 885009 17 8850...
1,00087a6bd4dc_02.jpg,873779 4 875695 7 877612 9 879528 12 881267 15...
2,00087a6bd4dc_03.jpg,864300 9 866217 13 868134 15 870051 16 871969 ...
3,00087a6bd4dc_04.jpg,879735 20 881650 26 883315 92 883564 30 885208...
4,00087a6bd4dc_05.jpg,883365 74 883638 28 885262 119 885550 34 88716...


In [11]:
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

In [12]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [13]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image

In [14]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


In [15]:
def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask


In [16]:
all_imgs  = {}
all_masks = {}
for id in ids_train:
    img  = cv2.imread('data/train/{}.jpg'.format(id))
    img  = cv2.resize(img, (input_size, input_size))
    mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (input_size, input_size))
    all_imgs[id]  = img
    all_masks[id] = mask

In [17]:
import random

In [26]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(list(ids_train_split), len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = all_imgs[id]
                mask = all_masks[id]
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomShiftScaleRotate(img, mask,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.1, 0.1),
                                                   rotate_limit=(-0, 0))
                img, mask = randomHorizontalFlip(img, mask)
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch


In [27]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch.values:
                img  = all_imgs[id]
                mask = all_masks[id]
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch



In [ ]:
callbacks = [EarlyStopping(monitor='val_dice_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_loss',
                             filepath='weights/best_weights_512.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max')]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
1095s - loss: 0.6683 - dice_loss: 0.6175 - val_loss: 1.2824 - val_dice_loss: 0.2345
Epoch 2/50
1067s - loss: 0.5082 - dice_loss: 0.6943 - val_loss: 0.4517 - val_dice_loss: 0.7234
Epoch 3/50
1067s - loss: 0.4092 - dice_loss: 0.7463 - val_loss: 0.3579 - val_dice_loss: 0.7762
Epoch 4/50


In [28]:
history.history

{'dice_loss': [0.61744884248154752,
  0.69425383410816988,
  0.74630680787182557,
  0.79495358329440213,
  0.83816823004504681,
  0.87534160611084699,
  0.9078067352789333,
  0.93272636519603125,
  0.95118491757999768,
  0.96488442769507521,
  0.9746827342879274,
  0.98127262747551358,
  0.98564540426034009,
  0.98859289618030521,
  0.9904549333621594,
  0.99160921377397582,
  0.99242575801268251,
  0.99290880817160265,
  0.99320635268377733,
  0.99342375973225816,
  0.99358152025161861,
  0.99366723606568885,
  0.99383761367282353,
  0.99392326865207947,
  0.99397791064634955,
  0.99405429621000552,
  0.99410069331490147,
  0.99414823125562735,
  0.99423592283919049,
  0.99427576533699502,
  0.99431950000345848,
  0.99431004966506209,
  0.99442397701652396,
  0.9944121579866152,
  0.99448055310097017,
  0.9944686188744678,
  0.99454326524195569,
  0.99459396492351182,
  0.99482175782505944,
  0.9948549615956056,
  0.99490524348814302,
  0.99490636921046116,
  0.99494183781105994,
  0.

In [28]:
model.load_weights('./weights/best_weights_512.hdf5')

In [29]:
 model.compile(optimizer=RMSprop(lr=0.00001),
                  loss=bce_dice_loss,
                  metrics=[dice_loss])

In [30]:
max_epochs = 10

In [31]:
callbacks = [EarlyStopping(monitor='val_dice_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_loss',
                             filepath='weights/best_weights_512_2.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max')]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/10
1058s - loss: 0.0097 - dice_loss: 0.9949 - val_loss: 0.0090 - val_dice_loss: 0.9953
Epoch 2/10
1035s - loss: 0.0097 - dice_loss: 0.9950 - val_loss: 0.0090 - val_dice_loss: 0.9953
Epoch 3/10
1036s - loss: 0.0096 - dice_loss: 0.9950 - val_loss: 0.0090 - val_dice_loss: 0.9953
Epoch 4/10
1036s - loss: 0.0096 - dice_loss: 0.9950 - val_loss: 0.0090 - val_dice_loss: 0.9953
Epoch 5/10
1037s - loss: 0.0096 - dice_loss: 0.9950 - val_loss: 0.0090 - val_dice_loss: 0.9954
Epoch 6/10

Epoch 00005: reducing learning rate to 9.999999747378752e-07.
1037s - loss: 0.0096 - dice_loss: 0.9950 - val_loss: 0.0090 - val_dice_loss: 0.9954
Epoch 7/10
1036s - loss: 0.0095 - dice_loss: 0.9950 - val_loss: 0.0089 - val_dice_loss: 0.9954
Epoch 8/10
1036s - loss: 0.0096 - dice_loss: 0.9950 - val_loss: 0.0089 - val_dice_loss: 0.9954
Epoch 9/10
1037s - loss: 0.0096 - dice_loss: 0.9950 - val_loss: 0.0089 - val_dice_loss: 0.9954
Epoch 10/10

Epoch 00009: reducing learning rate to 9.999999974752428e-08.
1036s -

In [33]:
history.history

{'dice_loss': [0.99494539534430537,
  0.99495211432548358,
  0.99498290312670956,
  0.99498643227818562,
  0.99500851918204114,
  0.99501778798255636,
  0.99503850734790367,
  0.99501852104646271,
  0.99501814027969027,
  0.99505049901160914],
 'loss': [0.0097230667362781364,
  0.009700357252205619,
  0.0096289480777791443,
  0.0096364085770759,
  0.0095760124186351353,
  0.009560881669493506,
  0.0095138971526770692,
  0.0095830219111311503,
  0.0095739817498503507,
  0.0095085102609411363],
 'lr': [9.9999997e-06,
  9.9999997e-06,
  9.9999997e-06,
  9.9999997e-06,
  9.9999997e-06,
  9.9999997e-06,
  1e-06,
  1e-06,
  1e-06,
  1e-06],
 'val_dice_loss': [0.9953225075847274,
  0.99534502736711783,
  0.99534967383607664,
  0.99533991846448078,
  0.99535552820662854,
  0.99536956269989318,
  0.99537191866423147,
  0.99537215251585354,
  0.99537093185020098,
  0.99537680366887094],
 'val_loss': [0.0090009434503633048,
  0.0089679044709780591,
  0.0089681455334886355,
  0.0089795740764068484

# Predictions

In [34]:
from tqdm import tqdm

In [35]:
df_test = pd.read_csv('data/sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

In [36]:
names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))

In [37]:
# https://www.kaggle.com/stainsby/fast-tested-rle
def run_length_encode(mask):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    inds = mask.flatten()
    runs = np.where(inds[1:] != inds[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle = ' '.join([str(r) for r in runs])
    return rle


rles = []

model.load_weights(filepath='weights/best_weights_512_2.hdf5')


In [38]:
val_batch_size = 32

In [39]:
print('Predicting on {} samples with batch_size = {}...'.format(len(ids_test), val_batch_size))
for start in tqdm(range(0, len(ids_test), val_batch_size)):
    x_batch = []
    end = min(start + val_batch_size, len(ids_test))
    ids_test_batch = ids_test[start:end]
    for id in ids_test_batch.values:
        img = cv2.imread('data/test/{}.jpg'.format(id))
        img = cv2.resize(img, (input_size, input_size))
        x_batch.append(img)
    x_batch = np.array(x_batch, np.float32) / 255
    preds = model.predict_on_batch(x_batch)
    preds = np.squeeze(preds, axis=3)
    for pred in preds:
        prob = cv2.resize(pred, (orig_width, orig_height))
        mask = prob > threshold
        rle = run_length_encode(mask)
        rles.append(rle)

  0%|          | 0/3127 [00:00<?, ?it/s]

Predicting on 100064 samples with batch_size = 32...


100%|██████████| 3127/3127 [3:40:15<00:00,  4.19s/it]  


In [41]:
print("Generating submission file...")
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/submission3.csv.gz', index=False, compression='gzip')

Generating submission file...
